# **Amazon Reviews: Pet Products ETL**

## Setup

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [2]:
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-01 18:15:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.23MB/s    in 0.2s    

2022-06-01 18:15:04 (5.23 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



## Extract

In [3]:
# Extract dataset from S3
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PetsCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Load dataset into a DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

## Transform

In [5]:
# Drop duplicate and/or incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

2643619
2643241
2643241


In [6]:
# Examine the schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Create a new DataFrame for Review IDs
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
print(review_id_df.count())
review_id_df = review_id_df.dropna()
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates()
print(review_id_df.count())
review_id_df.show(10)

2643241
2643241
2643241
+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R102OI838NN3Z7|   11056168|B000YDUAMQ|     180607962| 2014-01-18|
|R11UGDI1AMPER6|   52510120|B005AP3B88|     528349565| 2013-11-29|
|R12ZNC8UWAUPHK|   44422020|B00C1G7DBU|     233306731| 2015-03-15|
|R13UL58ATVRTFH|    1739224|B00CUIPVD6|     156511987| 2014-09-19|
|R16VQ553B2N91O|   35931185|B00798FYR0|     376800232| 2014-04-16|
|R17K44LG0YMN28|   15083271|B003FWOIG2|     216603062| 2011-10-02|
|R17LI3CIEOOJ8W|   36849613|B000F2HE9S|     110149424| 2014-08-08|
|R18MZGHA7SZ7YX|   37288160|B00H7Q3QVK|     598736025| 2014-04-18|
|R19QCXMS8PDMFH|   46829871|B00E4T5ERW|     290909737| 2015-03-23|
|R1GFYWJ2JNPR7I|   10643625|B000FSN0A4|     935408697| 2011-09-20|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [8]:
# Adjust data types to match DB
from pyspark.sql.types import * 
review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# Create a new DataFrame for Products
products_df = df.select(["product_id", "product_title"])
print(products_df.count())
products_df = products_df.dropna()
print(products_df.count())
products_df = products_df.dropDuplicates()
print(products_df.count())
products_df.show(10)

2643241
2643241
239307
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00AT910JM|PetSafe Drinkwell...|
|B00134HSYS|Special Edition P...|
|B00CRW2Z8E|Merrick Grain Fre...|
|B00JZQ28PU|Nature's Perfect ...|
|B003ZYF1LG|K&H Manufacturing...|
|B004OA5ZLG|Felisyl Immune Sy...|
|B004UM9UXW|Authentic Poochie...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B008EXDOT4|BLUE Basics Limit...|
|B0056QYEO6|Top Performance N...|
+----------+--------------------+
only showing top 10 rows



In [10]:
# Create a new DataFrame for Customers
customers_df = df.select("customer_id")
customers_df = customers_df.groupby("customer_id").count()
customers_df = customers_df.withColumnRenamed("count", "customer_count")
print(customers_df.count())
customers_df.show(10)

1414974
+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   20214846|            15|
|   31120744|             4|
|   20223672|             2|
|   39043742|             1|
|   15231334|             9|
|   44700871|             2|
|   26617857|             7|
|   51750322|             1|
|   42655488|             1|
|   48278478|             1|
+-----------+--------------+
only showing top 10 rows



In [11]:
# Adjust data types to match DB
customers_df = customers_df.withColumn("customer_id",customers_df["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers_df["customer_count"].cast(IntegerType()))
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [12]:
# Create a new DataFrame for Vines
vines_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine" ])
print(vines_df.count())
vines_df = vines_df.dropna()
print(vines_df.count())
vines_df = vines_df.dropDuplicates()
print(vines_df.count())
vines_df.show(10)

2643241
2643241
2643241
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R10936L9CMYJQ|          5|            0|          0|   N|
|R11CTDQV5BTUVL|          5|            0|          0|   N|
|R12ZNC8UWAUPHK|          1|            0|          0|   N|
|R1C31T915J1TJS|          4|            1|          2|   N|
|R1C4083EHMXKGB|          3|            1|          3|   N|
|R1CT9JBT4XPPBL|          4|            0|          0|   N|
|R1D2SJ7SP476P0|          5|            0|          0|   N|
|R1DHF63D6RTVFX|          1|            5|          6|   N|
|R1HOKS9YJZIF9F|          5|            0|          0|   N|
|R1JQO5CT54TWEO|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [13]:
# Adjust data types to match DB
vines_df = vines_df.withColumn("star_rating",vines_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vines_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vines_df["total_votes"].cast(IntegerType()))
vines_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



## Load

In [14]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://nicolefirstdatabase.cmrhlgsp7vsd.us-east-2.rds.amazonaws.com:5432/awsdatabase"
config = {"user":"postgres",
          "password": "",
          "driver":"org.postgresql.Driver"}


In [15]:
# Write DataFrame to RDS
review_id_df.write.jdbc(url = jdbc_url, table = "review_id_table", mode = mode, properties = config)

In [16]:
products_df.write.jdbc(url = jdbc_url, table = "products", mode = mode, properties = config)

In [17]:
customers_df.write.jdbc(url = jdbc_url, table = "customers", mode = mode, properties = config)

In [18]:
vines_df.write.jdbc(url = jdbc_url, table = "vine_table", mode = mode, properties = config)